<a href="https://colab.research.google.com/github/nguyenkimthach/Optimization_Binary_Neural_Network/blob/main/CNN_GTSRB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BinaryNet on GTSRB

In [ ]:
!pip install larq
!pip install tensorflow==2.7.2

In [ ]:
# Fundamental classes
import tensorflow as tf
import matplotlib.pyplot as plt
import larq as lq
import numpy as np
from tensorflow.keras.models import Sequential, load_model
#from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout


# 2. Loading Dataset (Traffic Signs Images)

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

import cv2
from PIL import Image
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Setting variables for later use
data = []
labels = []
classes = 43
cur_path = os.getcwd()

# Retrieving the images and their labels
for i in range(classes):
    path = os.path.join('/content/drive/MyDrive/gtsrb-german-traffic-sig/','Train',str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

# Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

# 3. Data Splitting and conversion

In [ ]:
# Checking data shape
print(data.shape, labels.shape)

# Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Displaying the shape after the split
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

# 4. Creating and Compiling the Model

In [ ]:
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D,Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from keras.models import Sequential

model = Sequential()

# filters 32, size of filters (3,3)
model.add(Conv2D(16, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(BatchNormalization(axis=-1)) #BatchNormalization normalizes the matrix after it is been through a convolution layer so that the scale of each dimension remains the same
model.add(Activation('relu'))

model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) # act as an input to the Dense layers.

# Fully connected layer
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(43))
model.add(BatchNormalization())
model.add(Activation('softmax')) # Softmax activation enables us to calculate the output based on the probabilities

In [ ]:
# Compilation of the model
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(
     tf.keras.optimizers.Adam(learning_rate=0.01),
     loss="categorical_crossentropy",
     metrics=["accuracy"],
)

# 5. Training the Model

In [ ]:
# Training the Model classification-96-cnn
with tf.device('/GPU:0'):
    epochs = 30
    history = model.fit(
        X_train,
        y_train,
        batch_size=32,
        epochs=epochs,
        validation_data=(X_test, y_test)
)

    test_loss, test_acc = model.evaluate(X_test, y_test)

# 6. Visualizing the performance of the Model during Training Phase

In [ ]:
# Plotting performance graphs
#plt.figure(0)
paras1 = history.history['accuracy']
paras1.insert(0,0.8)
plt.plot(history.history['accuracy'], label='training accuracy')
#plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
#plt.figure(1)
paras = history.history['loss']
paras.insert(0,1.2)
plt.plot(history.history['loss'], label='training loss')
#plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(f"Test accuracy {test_acc * 100:.2f} %")

# 7. Loading Test Dataset and Evaluating the Model

In [ ]:
# testing accuracy on test dataset
from sklearn.metrics import accuracy_score

# Importing the test dataset
y_test = pd.read_csv('/content/drive/MyDrive/gtsrb-german-traffic-sig/Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

# Retreiving the images
with tf.device('/GPU:0'):
    for img in imgs:
        image = Image.open('/content/drive/MyDrive/gtsrb-german-traffic-sig/'+img)
        image = image.resize([30, 30])
        data.append(np.array(image))

X_test=np.array(data)

with tf.device('/GPU:0'):
    pred = np.argmax(model.predict(X_test), axis=-1)

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))


# 8. Savinng the Model

In [ ]:
# Saving the Model
model.save('traffic_classifier.h5')